In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.stats import multivariate_normal
from numpy.linalg import eigvals, eigh
from IPython.display import Math, display
import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive_output

In [4]:
# Fungsi untuk menampilkan parameter dalam format LaTeX
def display_parameters(mu, sigma):
    # Mengonversi vektor mean dan matriks kovarians ke format LaTeX
    mu_latex = vector_to_latex(mu)
    sigma_latex = matrix_to_latex(sigma)
    
    # Menampilkan vektor mean dan matriks kovarians
    display(Math(r"\text{Vektor mean } \mu = " + mu_latex))
    display(Math(r"\text{Matriks kovarians } \Sigma = " + sigma_latex))
    
    # Hitung dan tampilkan eigenvalue dan eigenvector
    calculate_and_display_eigenvalues_vectors(sigma)

# Fungsi bantu untuk memformat vektor dalam LaTeX
def vector_to_latex(vector):
    """Memformat vektor untuk ditampilkan dalam LaTeX."""
    latex = r'\begin{bmatrix}' + r' \\ '.join(f'{num:.2f}' for num in vector) + r'\end{bmatrix}'
    return latex

# Fungsi bantu untuk memformat matriks dalam LaTeX
def matrix_to_latex(matrix):
    """Memformat matriks untuk ditampilkan dalam LaTeX."""
    latex = r'\begin{bmatrix}'
    rows, cols = matrix.shape
    for i in range(rows):
        row_items = ' & '.join(f'{num:.2f}' for num in matrix[i])
        latex += row_items
        if i < rows - 1:
            latex += r' \\ '  # Tambahkan line break
    latex += r'\end{bmatrix}'
    return latex

# Fungsi untuk menghitung dan menampilkan eigenvalue dan eigenvector
def calculate_and_display_eigenvalues_vectors(sigma):
    # Ekstraksi elemen dari matriks kovarians
    a, b = sigma[0, 0], sigma[0, 1]
    c, d = sigma[1, 0], sigma[1, 1]
    
    # Pastikan matriks kovarians simetris
    if not np.allclose(sigma, sigma.T):
        print("Matriks kovarians harus simetris.")
        return
    if not np.all(eigvals(sigma) > 0):
        print("Matriks kovarians harus positif definit.")
        return
    
    # Tampilkan persamaan karakteristik
    display(Math(r"\text{Mencari eigenvalue } \lambda \text{ dari persamaan karakteristik:}"))
    display(Math(r"\det(\Sigma - \lambda I) = 0"))
    
    # Tampilkan perhitungan determinan
    display(Math(r"\det\left(\begin{bmatrix} a - \lambda & b \\ b & d - \lambda \end{bmatrix}\right) = 0"))
    
    # Substitusi nilai
    display(Math(rf"\det\left(\begin{{bmatrix}} {a:.2f} - \lambda & {b:.2f} \\ {b:.2f} & {d:.2f} - \lambda \end{{bmatrix}}\right) = 0"))
    
    # Hitung determinan dan tampilkan persamaan kuadratik
    # Determinan: (a - λ)(d - λ) - b^2 = 0
    # Ekspansi: λ^2 - (a + d)λ + (ad - b^2) = 0
    trace = a + d
    determinant = a*d - b**2
    display(Math(rf"({a:.2f} - \lambda)({d:.2f} - \lambda) - ({b:.2f})^2 = 0"))
    display(Math(rf"\lambda^2 - {trace:.2f}\lambda + ({a:.2f} \times {d:.2f} - {b:.2f}^2) = 0"))
    display(Math(rf"\lambda^2 - {trace:.2f}\lambda + {determinant:.2f} = 0"))
    
    # Selesaikan persamaan kuadratik untuk λ
    # λ = [trace ± sqrt(trace^2 - 4 * determinant)] / 2
    discriminant = trace**2 - 4 * determinant
    sqrt_discriminant = np.sqrt(discriminant)
    lambda1 = (trace + sqrt_discriminant) / 2
    lambda2 = (trace - sqrt_discriminant) / 2
    display(Math(rf"\lambda = \frac{{{trace:.2f} \pm \sqrt{{({trace:.2f})^2 - 4 \times {determinant:.2f}}}}}{{2}}"))
    display(Math(rf"\lambda_1 = {lambda1:.3f}, \quad \lambda_2 = {lambda2:.3f}"))
    
    # Sekarang hitung eigenvector untuk setiap eigenvalue
    eigenvalues = np.array([lambda1, lambda2])
    eigenvectors = []
    for idx, eigenvalue in enumerate(eigenvalues):
        display(Math(rf"\text{{Mencari eigenvector untuk }} \lambda_{idx+1} = {eigenvalue:.3f}"))
        # Selesaikan (Sigma - lambda*I)v = 0
        matrix = sigma - eigenvalue * np.eye(2)
        display(Math(rf"(\Sigma - \lambda I) = \begin{{bmatrix}} {a:.2f} - {eigenvalue:.3f} & {b:.2f} \\ {b:.2f} & {d:.2f} - {eigenvalue:.3f} \end{{bmatrix}} = \begin{{bmatrix}} {matrix[0,0]:.3f} & {matrix[0,1]:.3f} \\ {matrix[1,0]:.3f} & {matrix[1,1]:.3f} \end{{bmatrix}}"))
        # Karena sistem memiliki derajat kebebasan 1, kita dapat menetapkan salah satu variabel dan menyelesaikan yang lain
        # Misalkan v = [v1, v2]
        # (matrix) * [v1, v2] = [0, 0]
        # Persamaan: (a - λ)v1 + b*v2 = 0
        # Kita dapat mengekspresikan v2 dalam fungsi v1 atau sebaliknya
        if abs(matrix[0, 1]) > 1e-8:
            v1 = 1
            v2 = -matrix[0, 0]/matrix[0, 1]
        elif abs(matrix[1, 0]) > 1e-8:
            v2 = 1
            v1 = -matrix[1, 1]/matrix[1, 0]
        else:
            # Jika matrix[0,1] dan matrix[1,0] keduanya nol
            v1 = 1
            v2 = 0
        eigenvector = np.array([v1, v2])
        # Normalisasi eigenvector
        norm = np.linalg.norm(eigenvector)
        if norm != 0:
            eigenvector_normalized = eigenvector / norm
        else:
            eigenvector_normalized = eigenvector
        eigenvectors.append(eigenvector_normalized)
        display(Math(rf"\text{{Eigenvector (tidak dinormalisasi)}}: v = \begin{{bmatrix}} {eigenvector[0]:.3f} \\ {eigenvector[1]:.3f} \end{{bmatrix}}"))
        display(Math(rf"\text{{Eigenvector (dinormalisasi)}}: \hat{{v}} = \begin{{bmatrix}} {eigenvector_normalized[0]:.3f} \\ {eigenvector_normalized[1]:.3f} \end{{bmatrix}}"))
    # Tidak perlu mengembalikan nilai, karena kita hanya menampilkan

# Fungsi untuk menggambar elips yang merepresentasikan interval kepercayaan
def plot_ellipse(ax, mean, cov, n_std=2.0, **kwargs):
    """
    Menggambar elips yang merepresentasikan matriks kovarians berpusat di mean.
    """
    eigenvalues, eigenvectors = eigh(cov)
    order = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[order]
    eigenvectors = eigenvectors[:, order]
    angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
    width, height = 2 * n_std * np.sqrt(eigenvalues)
    ell = Ellipse(xy=mean, width=width, height=height, angle=angle, **kwargs)
    ax.add_patch(ell)

# Fungsi untuk menggambar kontur kepadatan
def plot_density_contours(mu, sigma, n_std):
    """Menggambar kontur kepadatan untuk distribusi Gaussian bivariat."""
    fig, ax = plt.subplots(figsize=(8, 6))
    # Membuat grid dan distribusi normal multivariat
    x_lim = [mu[0] - n_std * 4 * np.sqrt(sigma[0,0]), mu[0] + n_std * 4 * np.sqrt(sigma[0,0])]
    y_lim = [mu[1] - n_std * 4 * np.sqrt(sigma[1,1]), mu[1] + n_std * 4 * np.sqrt(sigma[1,1])]
    x, y = np.mgrid[x_lim[0]:x_lim[1]:.05, y_lim[0]:y_lim[1]:.05]
    pos = np.dstack((x, y))
    try:
        rv = multivariate_normal(mu, sigma)
    except np.linalg.LinAlgError:
        print("Matriks kovarians tidak positif definit.")
        return
    # Menggambar garis kontur
    ax.contourf(x, y, rv.pdf(pos), levels=50, cmap='viridis')
    ax.set_xlim(x_lim)
    ax.set_ylim(y_lim)
    ax.set_xlabel('Sumbu X')
    ax.set_ylabel('Sumbu Y')
    # Menggambar elips kepercayaan
    label = f'Elips {n_std}σ'
    plot_ellipse(ax, mu, sigma, n_std=n_std, edgecolor='red', facecolor='none', linewidth=2, label=label)
    ax.legend()
    plt.show()

# Fungsi untuk menangani input pengguna dan menggambar plot
def interactive_plot(mean_x, mean_y, sigma_xx, sigma_xy, sigma_yy, n_std):
    mu = np.array([mean_x, mean_y])
    sigma = np.array([[sigma_xx, sigma_xy], [sigma_xy, sigma_yy]])
    
    # Validasi matriks kovarians
    if not np.allclose(sigma, sigma.T):
        print("Matriks kovarians harus simetris.")
        return
    if not np.all(eigvals(sigma) > 0):
        print("Matriks kovarians harus positif definit.")
        return
    
    # Menampilkan parameter dan menghitung eigenvalues/eigenvectors
    display_parameters(mu, sigma)
    # Menggambar kontur kepadatan
    plot_density_contours(mu, sigma, n_std)

In [6]:
# Membuat widget interaktif untuk input pengguna
mean_x_widget = widgets.FloatText(value=0.0, description='Mean x:', step=0.1)
mean_y_widget = widgets.FloatText(value=0.0, description='Mean y:', step=0.1)
sigma_xx_widget = widgets.FloatText(value=1.0, description='σ_xx:', step=0.1)
sigma_xy_widget = widgets.FloatText(value=0.8, description='σ_xy:', step=0.1)
sigma_yy_widget = widgets.FloatText(value=1.0, description='σ_yy:', step=0.1)

# Widget untuk memilih interval kepercayaan
confidence_widget = widgets.Dropdown(
    options=[('68% (1σ)', 1.0), ('95% (2σ)', 2.0), ('99.7% (3σ)', 3.0)],
    value=2.0,
    description='Interval Kepercayaan:',
    style={'description_width': 'initial'}
)

# Mengatur tata letak widget
input_widgets = VBox([
    HBox([mean_x_widget, mean_y_widget]),
    HBox([sigma_xx_widget, sigma_xy_widget, sigma_yy_widget]),
    confidence_widget
])

In [8]:
# Menampilkan widget
display(input_widgets)

# Membuat output interaktif
out = widgets.interactive_output(
    interactive_plot,
    {
        'mean_x': mean_x_widget,
        'mean_y': mean_y_widget,
        'sigma_xx': sigma_xx_widget,
        'sigma_xy': sigma_xy_widget,
        'sigma_yy': sigma_yy_widget,
        'n_std': confidence_widget
    }
)

# Menampilkan output
display(out)

Output()